# RAG - How to query

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])
print(OPENAI_API_KEY[:10])

cwzujlbgtk
L1FUbzRPaU
sk-proj-iu


## Connect to Weaviate

In [3]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    headers = {
        "X-OpenAI-Api-Key": OPENAI_API_KEY
    },

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

### Start with (R) - Retrieval

In [4]:
wiki = client.collections.get("Wiki")

response = wiki.query.near_text(
    query="How do planes fly",
    limit=5,
    return_properties=["text", "title"]
)

for item in response.objects:
    print(item.properties)

{'text': "The wings are the most important part because they are what makes the plane fly. The wings create a force called lift that goes against gravity which makes the plane get off the ground. When air flows around the wing (which happens when the plane moves forward) the wing pushes air down, which in turn pushes the plane up. Lift can also be explained using Bernoulli's principle; since wings are designed so that air goes faster on the top of the wing than on the bottom, the higher pressure on the bottom will push the plane up.  Some wings have panels on the back called flaps. Flaps can be extended to make the wing bigger. This makes the wing make more lift, but also more drag. This lets the plane fly slower without stalling, so flaps are often used during takeoff and landing.", 'title': 'Fixed-wing aircraft'}
{'text': 'A wooden plane is entirely wood except for the blade. The iron is held into the plane with a wooden wedge, and is adjusted by striking the plane with a hammer.', '

### Add (AG) - augmented generation - to make full RAG

#### Single Prompt

> Generate a response per **retrieved** object.

In [5]:
# Let's add some colour to our lives :)
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

In [6]:
from weaviate.classes.generate import GenerativeConfig

wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    # TODO: add GenerativeConfig with OpenAI and "gpt-4o-mini"
    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
    
    # TODO: add a single prompt "Explain what this is about? {text}"
    single_prompt="Explain what this is about? {text}"
)

for item in response.objects:
    print(f"{BLUE}=== Source ===")
    print(item.properties)

    print(f"{PURPLE}=== Generated Response ===")
    print(item.generative.text)
    print("\n")

=== Source ===
{'text': "The wings are the most important part because they are what makes the plane fly. The wings create a force called lift that goes against gravity which makes the plane get off the ground. When air flows around the wing (which happens when the plane moves forward) the wing pushes air down, which in turn pushes the plane up. Lift can also be explained using Bernoulli's principle; since wings are designed so that air goes faster on the top of the wing than on the bottom, the higher pressure on the bottom will push the plane up.  Some wings have panels on the back called flaps. Flaps can be extended to make the wing bigger. This makes the wing make more lift, but also more drag. This lets the plane fly slower without stalling, so flaps are often used during takeoff and landing.", 'title': 'Fixed-wing aircraft', 'wiki_id': '20231101.simple_4700_4', 'url': 'https://simple.wikipedia.org/wiki/Fixed-wing%20aircraft'}
=== Generated Response ===
The passage explains the rol

#### Grouped Task

> Generate one response based on all **retrieved** objects.

In [7]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    # auto_limit=1,
    limit=5,

    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),

    grouped_task="Explain, how do planes fly? Please only use the provided content."
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
Planes fly primarily due to their wings, which create a force known as lift that counteracts gravity, allowing the plane to ascend. As the plane moves forward, air flows around the wings. The design of the wings is such that air travels faster over the top of the wing than underneath, resulting in lower pressure on top and higher pressure beneath, which pushes the plane upwards. 

Additionally, some wings are equipped with flaps on their trailing edges. These flaps can be extended to increase the wing's surface area, which enhances lift, albeit at the cost of increased drag. This feature allows the plane to fly at slower speeds without stalling, making flaps particularly useful during takeoff and landing.

Furthermore, aircraft utilize propellers to move air over their wings, facilitating flight by enhancing lift. This combination of wing design and aerodynamics is essential for a plane's ability to fly.
=== Source ===
{'text': "The wings are the most importa

#### Specify which properties to use for grouped task

In [8]:
wiki = client.collections.get("Wiki")

response = wiki.generate.near_text(
    query="How do planes fly",
    auto_limit=1,
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
    grouped_properties=["text", "title"],

    generative_provider=GenerativeConfig.openai(
        model="gpt-4o-mini",
    ),
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
Planes fly primarily due to their wings, which are crucial for generating lift, a force that opposes gravity. As the plane moves forward, air flows around the wings, and the shape of the wing pushes air downward, resulting in an upward force on the plane. This lift can also be understood through Bernoulli's principle, where the design of the wings ensures that air moves faster over the top than beneath, creating lower pressure on top and higher pressure below, thus pushing the plane upwards.

Additionally, wings may have adjustable flaps on the back, which can be extended to increase the wing's surface area. This increase in size enhances lift but also raises drag, allowing the plane to fly at slower speeds without stalling. Flaps are commonly used during critical phases of flight, such as takeoff and landing.
=== Source ===
{'text': "The wings are the most important part because they are what makes the plane fly. The wings create a force called lift that goe

## Set default Generative model

In [9]:
from weaviate.classes.config import Reconfigure

wiki = client.collections.get("Wiki")

wiki.config.update(
    generative_config=Reconfigure.Generative.openai(
        model="gpt-4o-mini"  # Update the generative model
    )
)

> Try generative query without providing the model

In [10]:
response = wiki.generate.near_text(
    query="What african animals do we have info on. Please only list those provided in here.",
    auto_limit=1,
    
    grouped_task="Explain, how do planes fly? Please only use the provided content.",
)

print(f"{PURPLE}=== Generated Response ===")
print(response.generative.text)

print(f"{BLUE}=== Source ===")
for item in response.objects:
    print(item.properties)

=== Generated Response ===
I'm sorry, but the provided content does not contain any information about how planes fly. If you have any other materials or topics you'd like to discuss, please let me know!
=== Source ===
{'text': 'Lizards: Solomon Islands skinks, Madagascar giant day geckos, reticulated Gila monsters. blue tree monitors, African fat-tailed geckos, Razo Island skinks, Desert grassland whiptail lizards, and an ornate monitor.', 'title': 'Cincinnati Zoo and Botanical Garden', 'wiki_id': '20231101.simple_334964_22', 'url': 'https://simple.wikipedia.org/wiki/Cincinnati%20Zoo%20and%20Botanical%20Garden'}
{'text': 'The animals of the genus Camelus are also called Afro-Asiatic Camelids. The animals of the genus Lama and genus Vicugna are also called South American Camelids.', 'title': 'Camelid', 'wiki_id': '20231101.simple_31585_1', 'url': 'https://simple.wikipedia.org/wiki/Camelid'}
{'text': "Intelligence is not limited to humans. Many animals also show signs of intelligence: an

## Close the client

In [11]:
client.close()